In [12]:
# pip install insightface onnxruntime-gpu
# ! pip install git+https://github.com/InstantID/InstantID.git
# ! pip install onnxruntime-gpu opencv-python-headless insightface

! pip install git+https://github.com/s0md3v/roop.git

  Cloning https://github.com/s0md3v/roop.git to /tmp/pip-req-build-wmhka4ht
  Running command git clone --filter=blob:none --quiet https://github.com/s0md3v/roop.git /tmp/pip-req-build-wmhka4ht
  Resolved https://github.com/s0md3v/roop.git to commit f9fe11a8a9a0429f2f1f67095af9c71677002793
ERROR: git+https://github.com/s0md3v/roop.git does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


# Insight Face

In [8]:

%load_ext autoreload
%autoreload 2

from local_utils import *

# OpenCV와 dlib

In [15]:
import cv2
import numpy as np
import dlib
import os

def normalize_image(img):
    """이미지를 float32 타입으로 정규화"""
    if img.dtype != np.float32:
        img = img.astype(np.float32)
    return img / 255.0

def denormalize_image(img):
    """이미지를 uint8 타입으로 변환"""
    return np.uint8(np.clip(img * 255.0, 0, 255))

def get_landmarks(img, detector, predictor):
    """얼굴 랜드마크 검출"""
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    
    if len(faces) == 0:
        # 이미지 조정 후 다시 시도
        adjusted = cv2.convertScaleAbs(gray, alpha=1.5, beta=30)
        faces = detector(adjusted)
    
    if len(faces) == 0:
        print("얼굴을 찾을 수 없습니다.")
        return None
    
    # 가장 큰 얼굴 선택
    face = max(faces, key=lambda rect: rect.width() * rect.height())
    landmarks = predictor(gray, face)
    
    # 랜드마크 포인트를 numpy 배열로 변환
    points = np.array([[p.x, p.y] for p in landmarks.parts()], dtype=np.int32)
    return points

def get_delaunay_triangles(img_shape, points):
    """들로네 삼각분할"""
    rect = (0, 0, img_shape[1], img_shape[0])
    subdiv = cv2.Subdiv2D(rect)
    
    # 포인트 삽입 전 형식 변환
    for point in points:
        x, y = point.astype(int)
        subdiv.insert((int(x), int(y)))
    
    triangles = subdiv.getTriangleList()
    return np.array(triangles, dtype=np.int32)

def get_face_mask(img, landmarks):
    """얼굴 마스크 생성"""
    mask = np.zeros(img.shape[:2], dtype=np.float32)
    
    # 얼굴 윤곽선 포인트 그룹
    points = np.concatenate([
        landmarks[0:17],    # 턱선
        landmarks[17:27],   # 눈썹
        landmarks[27:36],   # 코
        landmarks[36:48],   # 눈
        landmarks[48:68]    # 입
    ])
    
    # 볼록 껍질 생성
    hull = cv2.convexHull(points)
    cv2.fillConvexPoly(mask, hull, 1)
    
    # 마스크 부드럽게 처리
    mask = cv2.GaussianBlur(mask, (15, 15), 0)
    return np.dstack([mask] * 3)

def face_swap(source_img, target_img):
    """얼굴 교체 메인 함수"""
    try:
        # dlib 초기화
        detector = dlib.get_frontal_face_detector()
        predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
        
        # 이미지 크기 조정
        target_height = 800
        target_width = int(target_height * target_img.shape[1] / target_img.shape[0])
        
        source_resized = cv2.resize(source_img, (target_width, target_height))
        target_resized = cv2.resize(target_img, (target_width, target_height))
        
        print("이미지 크기 조정 완료")
        
        # 랜드마크 검출
        source_landmarks = get_landmarks(source_resized, detector, predictor)
        target_landmarks = get_landmarks(target_resized, detector, predictor)
        
        if source_landmarks is None or target_landmarks is None:
            print("하나 이상의 이미지에서 얼굴을 찾을 수 없습니다.")
            return None
            
        print("랜드마크 검출 완료")
        
        # float32로 변환
        source_float = normalize_image(source_resized)
        target_float = normalize_image(target_resized)
        
        # 삼각분할
        triangles = get_delaunay_triangles(target_resized.shape, target_landmarks)
        print(f"삼각형 개수: {len(triangles)}")
        
        # 결과 이미지 준비
        result = target_float.copy()
        
        # 각 삼각형에 대해 워핑 수행
        for triangle in triangles:
            x1, y1, x2, y2, x3, y3 = triangle
            
            # 삼각형 점들
            pts1 = np.float32([[x1, y1], [x2, y2], [x3, y3]])
            pts2 = pts1.copy()
            
            # 삼각형 마스크
            mask = np.zeros((target_height, target_width), dtype=np.float32)
            cv2.fillConvexPoly(mask, pts2.astype(np.int32), 1.0)
            mask = cv2.GaussianBlur(mask, (3, 3), 0)
            mask_3d = np.dstack([mask] * 3)
            
            # 어파인 변환
            M = cv2.getAffineTransform(pts1, pts2)
            warped = cv2.warpAffine(source_float, M, (target_width, target_height),
                                  borderMode=cv2.BORDER_REFLECT)
            
            # 블렌딩
            result = result * (1 - mask_3d) + warped * mask_3d
        
        print("워핑 완료")
        
        # 얼굴 마스크 적용
        face_mask = get_face_mask(target_resized, target_landmarks)
        result = target_float * (1 - face_mask) + result * face_mask
        
        print("마스크 적용 완료")
        
        # 후처리
        result = cv2.addWeighted(target_float, 0.15, result, 0.85, 0)
        
        return denormalize_image(result)
        
    except Exception as e:
        print(f"얼굴 교체 중 오류 발생: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

def main():
    try:
        # 출력 디렉토리 생성
        output_dir = 'output_image'
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        # 이미지 로드
        source_path = 'input_image/gonsoo_fall.jpg'
        target_path = 'input_image/painter.jpeg'
        
        source_img = cv2.imread(source_path)
        target_img = cv2.imread(target_path)
        
        if source_img is None or target_img is None:
            print("이미지를 로드할 수 없습니다.")
            return
            
        print(f"소스 이미지 크기: {source_img.shape}")
        print(f"타겟 이미지 크기: {target_img.shape}")
        
        # 얼굴 교체 수행
        result = face_swap(source_img, target_img)
        
        if result is not None:
            output_path = os.path.join(output_dir, 'face_swap_result.jpg')
            success = cv2.imwrite(output_path, result)
            
            if success:
                print(f"결과가 성공적으로 저장되었습니다: {output_path}")
            else:
                print("이미지 저장에 실패했습니다.")
        else:
            print("얼굴 교체에 실패했습니다.")
            
    except Exception as e:
        print(f"에러가 발생했습니다: {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

소스 이미지 크기: (4032, 3024, 3)
타겟 이미지 크기: (1344, 768, 3)
이미지 크기 조정 완료
랜드마크 검출 완료
삼각형 개수: 114
워핑 완료
마스크 적용 완료
결과가 성공적으로 저장되었습니다: output_image/face_swap_result.jpg


# FaceSwap

In [1]:
import cv2
import numpy as np
from faceswap import FaceSwap

# FaceSwap 인스턴스 생성
face_swapper = FaceSwap()

# 두 개의 이미지 로드
# 이미지 로드
source_path = 'input_image/gonsoo_fall.jpg'
target_path = 'input_image/painter.jpeg'

source_image = cv2.imread(source_path)
target_image = cv2.imread(target_path)

# 얼굴 교체 수행
result = face_swapper.swap(source_image, target_image)

# 결과 저장
cv2.imwrite("result.jpg", result)

ModuleNotFoundError: No module named 'faceswap'